🎌 WhisperJAV - Google Colab Edition
Generate Accurate Subtitles for Japanese Adult Videos using Free GPU

📋 **Quick Start Guide**
1.  Run **Cell 1** → Connect Google Drive
2.  Run **Cell 2** → Install WhisperJAV
3.  Run **Cell 3** → Launch Control Panel
4.  Select Options → Click "Start Processing"
5.  Check Results → In your Drive's `WhisperJAV` folder

⚡ **Pro Tip:** Use `Runtime` → `Run all` to setup everything at once!

In [1]:
#@markdown Mount your Google Drive to access video files

from google.colab import drive
import os
from pathlib import Path
import IPython.display as display

print("🔄 Connecting to Google Drive...")
drive.mount('/content/drive')

# Create WhisperJAV folder if it doesn't exist
whisperjav_folder = Path('/content/drive/MyDrive/WhisperJAV')
whisperjav_folder.mkdir(exist_ok=True)

# Check for video files
media_extensions = ['.mp3',
                    '.wav',
                    '.opus',
                    '.m4a',
                    '.flac',
                    '.wmv',
                    '.mp4',
                    '.webm']
media_files = []
for ext in media_extensions:
    media_files.extend(whisperjav_folder.glob(f'*{ext}'))

print("✅ Google Drive connected!")
print(f"📁 WhisperJAV folder: {whisperjav_folder}")
print(f"🎬 Found {len(media_files)} video file(s)")

if not media_files:
    display.display(display.HTML("""
    <div style=\"background-color: #fff3cd; border: 1px solid #ffeaa7; border-radius: 5px; padding: 15px; margin: 10px 0;\">
        <h4 style=\"color: #856404; margin: 0;\">⚠️ No video files found!</h4>
        <p style=\"margin: 5px 0;\">Please upload your video files to:<br>
        <code style=\"background: #f8f9fa; padding: 2px 5px; border-radius: 3px;\">My Drive/WhisperJAV/</code></p>
        <p style=\"margin: 5px 0; font-size: 0.9em;\">Supported formats: MP4, AVI, MKV, MOV, WMV, FLV, WEBM</p>
    </div>
    """))
else:
    display.display(display.HTML("""
    <div style=\"background-color: #d4edda; border: 1px solid #c3e6cb; border-radius: 5px; padding: 15px; margin: 10px 0;\">
        <h4 style=\"color: #155724; margin: 0;\">✅ Ready to process!</h4>
        <p style=\"margin: 5px 0;\">Found videos in your WhisperJAV folder</p>
    </div>
    """))

🔄 Connecting to Google Drive...
Mounted at /content/drive
✅ Google Drive connected!
📁 WhisperJAV folder: /content/drive/MyDrive/WhisperJAV
🎬 Found 0 video file(s)


In [ ]:
#@markdown This will install all required packages (takes ~2-3 minutes)

import subprocess
import sys

print("📦 Installing WhisperJAV and dependencies...")
print("This may take a few minutes on first run...\n")

# Install system dependencies
print("1/4 Installing FFmpeg...")
subprocess.run(['apt', 'update', '-qq'], capture_output=True)
subprocess.run(['apt', 'install', '-y', '-qq', 'ffmpeg'], capture_output=True)

# Install PyTorch with CUDA support
print("2/4 Installing PyTorch with GPU support...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
               'torch', 'torchvision', 'torchaudio',
               '--index-url', 'https://download.pytorch.org/whl/cu118'],
               capture_output=False)

# Install WhisperJAV
print("3/4 Installing WhisperJAV...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'whisperjav'], capture_output=False)

# Install UI dependencies
print("4/4 Installing UI components...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'ipywidgets'], capture_output=False)

print("\n✅ Installation complete!")

# Check GPU availability
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🎮 GPU detected: {gpu_name}")
else:
    print("⚠️ No GPU detected - processing will be slower")

📦 Installing WhisperJAV and dependencies...
This may take a few minutes on first run...

1/4 Installing FFmpeg...
2/4 Installing PyTorch with GPU support...


In [ ]:
#@markdown Configure and run WhisperJAV with an easy-to-use interface

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess
import threading
import time
import json
from datetime import datetime
from pathlib import Path

# Define whisperjav_folder path again in case cells are run out of order
whisperjav_folder = Path('/content/drive/MyDrive/WhisperJAV')

# Custom CSS for better styling
display(HTML("""
<style>
.whisperjav-container {
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
}
.control-section {
    background: white;
    border: 1px solid #e1e4e8;
    border-radius: 8px;
    padding: 20px;
    margin: 10px 0;
    box-shadow: 0 1px 3px rgba(0,0,0,0.1);
}
.section-title {
    font-size: 18px;
    font-weight: 600;
    color: #24292e;
    margin-bottom: 15px;
    display: flex;
    align-items: center;
}
.section-title .emoji {
    margin-right: 8px;
    font-size: 24px;
}
.radio-group {
    margin: 10px 0;
}
.status-box {
    background: #f6f8fa;
    border: 1px solid #e1e4e8;
    border-radius: 6px;
    padding: 15px;
    margin: 10px 0;
    font-family: 'Consolas', 'Monaco', monospace;
    font-size: 13px;
    max-height: 300px;
    overflow-y: auto;
    white-space: pre-wrap;
}
</style>
"""))

# Create the main container
display(HTML('<div class="whisperjav-container">'))

# Title
display(HTML("""
<h1 style="text-align: center; color: #24292e; margin: 20px 0;">
    🎌 WhisperJAV Control Panel
</h1>
"""))

# Check for files again
media_extensions = ['.mp4', '.avi', '.mkv', '.mov', '.wmv', '.flv', '.webm']
media_files = []
for ext in media_extensions:
    media_files.extend(whisperjav_folder.glob(f'*{ext}'))

if not media_files:
    display(HTML("""
    <div class="control-section" style="background: #fff3cd; border-color: #ffeaa7;">
        <p style="color: #856404; text-align: center; margin: 0;">
            ⚠️ No video files found in /MyDrive/WhisperJAV/<br>
            Please upload videos and run Cell 1 & 3 again
        </p>
    </div>
    """))
else:
    # Speed Control Section
    display(HTML("""
    <div class="control-section">
        <div class="section-title">
            <span class="emoji">⚡</span>
            Speed Control
            <span style="font-size: 14px; color: #586069; margin-left: 10px;">
                (Quickie vs Less Mistakes)
            </span>
        </div>
    """))

    mode_widget = widgets.RadioButtons(
        options=[
            ('🚀 Faster - Quick processing, may miss some details', 'faster'),
            ('⚡ Fast - Good balance of speed and accuracy', 'fast'),
            ('🎯 Balanced - Best accuracy, takes more time', 'balanced')
        ],
        value='balanced',
        layout=widgets.Layout(width='100%')
    )
    display(mode_widget)
    display(HTML('</div>'))

    # Granularity Control Section
    display(HTML("""
    <div class="control-section">
        <div class="section-title">
            <span class="emoji">🔍</span>
            Granularity Control
            <span style="font-size: 14px; color: #586069; margin-left: 10px;">
                (Details vs Fewer Guesses)
            </span>
        </div>
    """))

    sensitivity_widget = widgets.RadioButtons(
        options=[
            ('🔥 Aggressive - Capture everything, more false positives', 'aggressive'),
            ('⚖️ Balanced - Optimal for most content', 'balanced'),
            ('🎯 Conservative - Only high confidence, may miss some', 'conservative')
        ],
        value='balanced',
        layout=widgets.Layout(width='100%')
    )
    display(sensitivity_widget)
    display(HTML('</div>'))

    # Language Section
    display(HTML("""
    <div class="control-section">
        <div class="section-title">
            <span class="emoji">🌐</span>
            Output Language
        </div>
    """))

    language_widget = widgets.RadioButtons(
        options=[
            ('🇯🇵 Japanese - Original transcription', 'japanese'),
            ('🇬🇧 English - Direct translation', 'english-direct')
        ],
        value='japanese',
        layout=widgets.Layout(width='100%')
    )
    display(language_widget)
    display(HTML('</div>'))

    # Enhancement Options
    display(HTML("""
    <div class="control-section">
        <div class="section-title">
            <span class="emoji">✨</span>
            Enhancement Options
        </div>
    """))

    adaptive_classification = widgets.Checkbox(
        value=False,
        description='Adaptive Classification',
        style={'description_width': 'initial'}
    )
    adaptive_audio = widgets.Checkbox(
        value=False,
        description='Adaptive Audio Enhancement',
        style={'description_width': 'initial'}
    )
    smart_post = widgets.Checkbox(
        value=False,
        description='Smart Postprocessing',
        style={'description_width': 'initial'}
    )

    display(widgets.VBox([adaptive_classification, adaptive_audio, smart_post]))
    display(HTML('</div>'))

    # Process Button and Status
    display(HTML('<div class="control-section" style="text-align: center;">'))

    process_button = widgets.Button(
        description='Start Processing',
        icon='play',
        button_style='success',
        layout=widgets.Layout(width='300px', height='50px')
    )

    # Status output
    status_output = widgets.Output()

    # Progress bar
    progress_bar = widgets.IntProgress(
        value=0, min=0, max=100, description='Progress:',
        bar_style='info', style={'bar_color': '#2ea44f'},
        layout=widgets.Layout(width='100%', display='none')
    )

    def process_videos(b):
        process_button.disabled = True
        process_button.description = 'Processing...'
        progress_bar.layout.display = 'flex'

        with status_output:
            clear_output()
            print(f"🚀 Starting WhisperJAV processing...")
            print(f"📁 Processing folder: {whisperjav_folder}")
            print(f"🎬 Found {len(media_files)} video(s)")
            print("-" * 50)

            # Build command
            cmd = [
                'python3', '-m', 'whisperjav',
                str(whisperjav_folder),
                '--mode', mode_widget.value,
                '--sensitivity', sensitivity_widget.value,
                '--subs-language', language_widget.value,
                '--output-dir', str(whisperjav_folder)
            ]

            # Add enhancement options
            if adaptive_classification.value: cmd.append('--adaptive-classification')
            if adaptive_audio.value: cmd.append('--adaptive-audio-enhancement')
            if smart_post.value: cmd.append('--smart-postprocessing')

            # Run process
            try:
                process = subprocess.Popen(
                    cmd,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    universal_newlines=True,
                    bufsize=1
                )

                for line in iter(process.stdout.readline, ''):
                    if line:
                        print(line.rstrip())
                        if "Step" in line or "Transcribing" in line:
                           progress_bar.value = min(progress_bar.value + 5, 95)

                process.wait()

                if process.returncode == 0:
                    progress_bar.value = 100
                    progress_bar.bar_style = 'success'
                    print("\n✅ Processing completed successfully!")

                else:
                    progress_bar.bar_style = 'danger'
                    print(f"\n❌ Processing failed with code: {process.returncode}")

            except Exception as e:
                progress_bar.bar_style = 'danger'
                print(f"\n❌ Error: {str(e)}")

            finally:
                process_button.disabled = False
                process_button.description = 'Start Processing'

    process_button.on_click(process_videos)

    display(process_button)
    display(progress_bar)
    display(HTML('</div>'))

    # Status Section
    display(HTML("""
    <div class="control-section">
        <div class="section-title">
            <span class="emoji">📊</span>
            Processing Status
        </div>
        <div class="status-box">
    """))
    display(status_output)
    display(HTML('</div></div>'))

display(HTML('</div>'))

In [ ]:
#@markdown Check your generated subtitles and download if needed

import os
import zipfile
from datetime import datetime
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path

whisperjav_folder = Path('/content/drive/MyDrive/WhisperJAV')
# List all SRT files
srt_files = list(whisperjav_folder.glob('*.srt'))
raw_subs_folder = whisperjav_folder / 'raw_subs'
raw_files = list(raw_subs_folder.glob('*')) if raw_subs_folder.exists() else []

print("📄 Subtitle Files Generated:")
print("-" * 50)

if srt_files:
    for srt in sorted(srt_files):
        size_kb = srt.stat().st_size / 1024
        print(f"✅ {srt.name} ({size_kb:.1f} KB)")
else:
    print("❌ No subtitle files found yet. Please run processing first.")

if raw_files:
    print(f"\n📁 Raw outputs: {len(raw_files)} file(s) in raw_subs/")

# Create download button
if srt_files or raw_files:
    print("\n" + "="*50)

    # Create a download widget
    download_button = widgets.Button(
        description='Download All Subtitles as ZIP',
        icon='download',
        button_style='primary',
        layout=widgets.Layout(width='300px')
    )
    download_output = widgets.Output()

    def download_all(b):
        with download_output:
            clear_output()
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            zip_name = f'WhisperJAV_subtitles_{timestamp}.zip'
            zip_path = Path(f'/content/{zip_name}')

            print(f"Creating {zip_name}...")
            with zipfile.ZipFile(zip_path, 'w') as zipf:
                # Add main SRT files
                for srt in srt_files:
                    zipf.write(srt, srt.name)

                # Add raw files
                for raw in raw_files:
                    zipf.write(raw, f'raw_subs/{raw.name}')

            files.download(str(zip_path))
            print(f"✅ Download started for {zip_name}")
            # os.remove(zip_path)

    download_button.on_click(download_all)
    display(download_button, download_output)

    # Show file preview option
    if srt_files:
        print("\n📖 Preview first few lines of latest subtitle:")
        latest_srt = max(srt_files, key=lambda x: x.stat().st_mtime)
        try:
            with open(latest_srt, 'r', encoding='utf-8') as f:
                preview = f.read(500)
                print(f"\n--- {latest_srt.name} ---")
                print(preview)
                if len(f.read()) > 0:
                    print("... (truncated)")
        except Exception as e:
            print(f"Could not read file: {e}")

In [ ]:
#@markdown View GPU and system status

import subprocess
print("🎮 GPU Info:")
!nvidia-smi --query-gpu=name,memory.total,memory.used,temperature.gpu --format=csv

print("\n" + "="*50)

# Disk usage
print("💾 Disk Usage:")
!df -h | grep -E "Filesystem|/content"

print("\n" + "="*50)

# WhisperJAV folder size
whisperjav_folder_path = '/content/drive/MyDrive/WhisperJAV'
result = subprocess.run(['du', '-sh', whisperjav_folder_path], capture_output=True, text=True)
print(f"📁 WhisperJAV folder size: {result.stdout.strip()}")

print("\n" + "="*50)

# Python packages
print("📦 Installed Packages:")
!pip list | grep -E "whisperjav|whisper|torch"

💡 **Tips & Troubleshooting**
---
**🚀 Performance Tips:**
- `Faster` mode: Best for quick previews (5-10x faster)
- `Balanced` mode: Recommended for final subtitles
- **GPU**: A T4 GPU (usually available in the free tier) provides a ~10x speedup over CPU.

**❓ Common Issues:**
- **No video files found?**
  - Ensure you've uploaded videos to: `My Drive/WhisperJAV/`
  - Re-run Cell 1 and Cell 3 to refresh the file list.
- **Out of memory?**
  - Try using the `faster` mode, which uses a smaller model.
  - Process shorter videos individually.
  - Restart the runtime: `Runtime` → `Restart runtime`
- **Session disconnected?**
  - Colab free tier has time limits and will disconnect due to inactivity.
  - Keep the browser tab active to prevent automatic disconnection.

**📞 Support:**
- Report issues: [GitHub Issues](https://github.com/meizhong986/whisperjav/issues)
- Documentation: [WhisperJAV Docs](https://github.com/meizhong986/whisperjav)

*Made with ❤️ for the JAV subtitle community*